# PDB-CCD TM

## Import Packages and Datasets

In [1]:
import qcportal
from qcportal.singlepoint import SinglepointDriver
from qcportal.singlepoint import QCSpecification
from qcportal.optimization import OptimizationSpecification

from qcportal.external import scaffold


In [ ]:
client = qcportal.PortalClient("https://api.qcarchive.molssi.org:443", cache_dir=".")
from qcfractal.snowflake import FractalSnowflake
snowflake = FractalSnowflake()
client = snowflake.client()

Process ForkProcess-2:
Traceback (most recent call last):
  File "/Users/jenniferclark/mamba/envs/qca-clean/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/jenniferclark/mamba/envs/qca-clean/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jenniferclark/mamba/envs/qca-clean/lib/python3.11/site-packages/qcfractal/snowflake.py", line 95, in _compute_process
    compute = ComputeManager(compute_config)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jenniferclark/mamba/envs/qca-clean/lib/python3.11/site-packages/qcfractalcompute/compute_manager.py", line 157, in __init__
    raise ValueError(f"Executor {ex} has no available programs")
ValueError: Executor local has no available programs


## Get Dataset, Update Description

In [3]:
dataset_name = "TM Benchmark Optimization Dataset Step 1 v0.0"
new_description = (
    "This dataset includes single metal complexes with: {'Pd', 'Fe', 'Zn', 'Mg', 'Cu', 'Li'}, and the non-metals: "
    "{'C', 'H', 'P', 'S', 'O', 'N', 'F', 'Cl', 'Br'}, with a complex charge of {-1,0,+1}. Additionally, there are some organic "
    "molecules for benchmarking purposes. These complexes are optimized using ROHF SOS-MP2 / def2-SVP and  ROHF / STO-3G in suit "
    "with the literature. The molecular weight min, mean, and max are 81, 445, and 1026, respectively. There are 81 unique "
    "molecules, each tmc is submitted with 3 different multiplicities to assess the spin state."
)
dataset = scaffold.from_json("scaffold.json.bz2", client)
#dataset = client.get_dataset("optimization", "TM Benchmark Optimization Dataset Step 1 v0.0")
dataset.description = new_description
dataset.extras["long_description"] = new_description
dataset.extras["submitter"] = "jaclark5"

## Delete Old Specification

In [4]:
dataset.delete_specification(f"scs-mp2/aug-cc-pvtz-dk", delete_records=True)

DeleteMetadata(error_description=None, errors=[(0, 'specification does not exist')], deleted_idx=[], n_children_deleted=0)

## Add New Specification

In [5]:
spec = OptimizationSpecification(
    program='geometric',
    qc_specification=QCSpecification(
        program='psi4',
        driver=SinglepointDriver.deferred,
        method="sos-mp2",
        basis="def2-svp",
        keywords={
            'maxiter': 500, 
            'scf_properties': ['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices', 
                               'lowdin_charges', 'mulliken_charges'],
            'function_kwargs': {'properties': ['dipole_polarizabilities']},
            'reference': 'rohf',
        },
    ),
    keywords={
        'tmax': 0.3,
        'check': 0,
        'qccnv': False,
        'reset': True,
        'trust': 0.1,
        'molcnv': False,
        'enforce': 0.0,
        'epsilon': 1e-05,
        'maxiter': 500,
        'coordsys': 'dlc',
        'constraints': {},
        'convergence_set': 'GAU',
    }, # keywords for geometric
)
name = f"sos-mp2/def2-svp"
dataset.add_specification(name=name, specification=spec)

InsertMetadata(error_description=None, errors=[], inserted_idx=[0], existing_idx=[])

In [6]:
spec = OptimizationSpecification(
    program='geometric',
    qc_specification=QCSpecification(
        program='psi4',
        driver=SinglepointDriver.deferred,
        method="hf",
        basis="sto-3g",
        keywords={
            'maxiter': 500, 
            'scf_properties': ['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices', 
                               'lowdin_charges', 'mulliken_charges'],
            'function_kwargs': {'properties': ['dipole_polarizabilities']},
            'reference': 'rohf',
        },
    ),
    keywords={
        'tmax': 0.3,
        'check': 0,
        'qccnv': False,
        'reset': True,
        'trust': 0.1,
        'molcnv': False,
        'enforce': 0.0,
        'epsilon': 1e-05,
        'maxiter': 500,
        'coordsys': 'dlc',
        'constraints': {},
        'convergence_set': 'GAU',
    }, # keywords for geometric
)
name = f"hf/sto-3g"
dataset.add_specification(name=name, specification=spec)

InsertMetadata(error_description=None, errors=[], inserted_idx=[0], existing_idx=[])

In [7]:
scaffold.to_json(dataset, compress=True, filename="scaffold_sos-mp2_hf.json")
#dataset.submit()